### Rutina para estimar HAND de alguna región 

In [2]:
%%time
# Ruta absoluta donde están guardados los dems que bajamos para toda la región del USGS. A menos que se quiera
#  procesar una región diferente a las planchas descargadas, este debería quedar por defecto
path_dems = '/opt/geociencia/coreclima/Compartidos/HAND_Region/Procesamiento_DEMs_prueba/1.DEMs_unzip/'

# Ruta donde van a quedar guardados los shapes procesados. El nombre debería ser un argumento de entrada, y debería
# crear la carpeta en caso que no exista el país
out_ruta = '/opt/geociencia/coreclima/Compartidos/HAND_Region/Mexico/'

# Coordenadas del dominio espacial que abarca todas las planchas que componen la región de interés.
# Lat_infer_izq, Lon_infer_izq, Lat_sup_der, Lon_sup_der 
dominio = '14,-117,32,-86'

# Extension del archivo que contiene los datos del DEM. Puede ser Tiff, o bil, o ascii, o cualquier formato de estos
extension = 'bil'

# Tamaño del pixel de salida del HAND. Se entra argumento para remuestrear y hacer la salida menos pesada para el
# procesamiento
size_pixel = 200

# ------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------


# Importar todas las  librerías

from osgeo import gdal, gdalconst
from pysheds.grid import Grid
import numpy as np
import fnmatch
import glob, os
import shutil
import re

# ------------------------------------------------------------------------------------------------------------------

# Preprarar la lista con el nombre de archivos a procesar
dominio = list(map(float, dominio.split(',')))
os.chdir(path_dems)
file_list = list()
for file in glob.glob('*.' + extension):
    file_list.append(file)
file_list.sort()

processlist = list()

for file in file_list:
    coord = re.findall(r'(\d+)_',file)
    coord = list(map(int, coord))
    lat = re.findall(r'(.)\d+_',file)[0]
    if lat == 's':
        coord[0]=-coord[0]
        
    if (dominio[0] <= coord[0] <= dominio[2] and dominio[1] <= -coord[1] <= dominio[3]):
        processlist.append(file)
        

# ------------------------------------------------------------------------------------------------------------------
# Remuestrear el raster de interés

for file in processlist:

    inputfile = path_dems+'/'+file
    input_ = gdal.Open(inputfile, gdalconst.GA_ReadOnly)
    inputProj = input_.GetProjection()
    inputTrans = input_.GetGeoTransform()
    bandinput = input_.GetRasterBand(1)  

    # Tamaño de celda en raster original (metros)
    cell_size=inputTrans[1]*111100
    # factor de relación entre tamaño viejo y tamaño nuevo
    factor = int(cell_size)/size_pixel
    f1 = factor*input_.RasterXSize
    ncell = int(f1)
    # factor corregido
    factor_corr = ncell/input_.RasterXSize
    # Resultado
    res = inputTrans[1]/factor_corr

    inputTrans = list(inputTrans)
    inputTrans[1] = res
    inputTrans[-1] = -res
    referenceTrans = tuple(inputTrans)
    x = ncell
    y = ncell


    bandreference = bandinput
    referenceProj = inputProj

    outputfile = out_ruta+file[:-4]+'_resize.tif'
    driver= gdal.GetDriverByName('GTiff')
    output = driver.Create(outputfile,x,y,1,bandreference.DataType)
    output.SetGeoTransform(referenceTrans)
    output.SetProjection(referenceProj)

    gdal.ReprojectImage(input_,output,inputProj,referenceProj,gdalconst.GRA_Bilinear)
    del output
    


# ------------------------------------------------------------------------------------------------------------------
# estimar el HAND
    
    # Cargar el DEM
    grid = Grid.from_raster(outputfile, data_name='dem')
    # Fill depressions in DEM
    grid.fill_depressions('dem', out_name='flooded_dem')
    # Resolve flats in DEM
    grid.resolve_flats('flooded_dem', out_name='inflated_dem')
    # Specify directional mapping
    dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
    # Compute flow directions
    grid.flowdir(data='inflated_dem', out_name='dir', dirmap=dirmap)
    # Calculate flow accumulation
    grid.accumulation(data='dir', dirmap=dirmap, out_name='acc')
    # Calcula la red de drenaje
    R_drain    = grid.view('acc')
    R_drain[R_drain<1000]  = 0
    R_drain[R_drain>=1000] = 1
    # Calcular Hand
    grid.compute_hand('dir', 'dem', R_drain, out_name='hand')
    grid.hand[grid.hand<0]  = 0
    # Guardar el Hand como raster
    grid.to_raster('hand',out_ruta+'Hand_'+file[:-4])
    os.remove(outputfile)

CPU times: user 15min 10s, sys: 19.6 s, total: 15min 30s
Wall time: 16min 33s
